# Librerias 

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import mean_squared_error



# Importaciones

In [2]:
train = pd.read_csv("train_no_dummy.csv")
test = pd.read_csv("test_no_dummy.csv")

In [3]:
train

,id,carat,cut,color,clarity,x,y,z,price
0,0,1.50,5,3,6,7.32,7.34,4.51,9.588
1,1,2.01,3,1,8,8.11,8.25,4.96,9.748
2,2,0.50,1,1,7,5.13,5.09,3.15,7.255
3,3,0.25,3,3,3,4.05,4.08,2.50,6.450
4,4,0.52,1,4,6,5.16,5.19,3.21,7.721
...,...,...,...,...,...,...,...,...,...
40450,40450,1.04,3,6,8,6.60,6.62,3.94,8.190
40451,40451,0.51,3,1,7,5.09,5.05,3.21,7.246
40452,40452,1.51,1,5,6,7.37,7.33,4.60,9.277
40453,40453,2.02,5,5,6,8.16,8.11,4.99,9.680


# Train test split

Para evitar problemas de desajuste o sobre ajuste dividimos el conjunto de datos de entrenamiento y de prueba par aposteriormente ajustar nuestro modelo y hacer prediciones sobre nuestro datos de prueba. 

In [4]:
X = train.drop(columns = ['price'])
y = train['price']

In [5]:
#train
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

#  Random Forest

Random Forest es un algritmo de aprendizaje supervisado que "aprende" por conjuntos muy util para hacer regresiones y clasificaciones.  

In [6]:
forest = RandomForestRegressor()
forest.fit(X_train,y_train)

RandomForestRegressor()

In [7]:
# Predictions
y_pred_train = forest.predict(X_train)
y_pred_test = forest.predict(X_test)

In [8]:
# Metrics
mse_train = mean_squared_error(y_pred_train,y_train)
mse_test = mean_squared_error(y_pred_test,y_test)
mse_train, mse_test

(0.0012822573195062441, 0.008872961837671485)

## Grid search

Grid search permite realizar multiples permutaciones y combinaciones para obtener la mejor combinacion entre los multiples parametros de para obtener el mejor modelo para realizar nuestras predicciones.

In [9]:
model = RandomForestRegressor()
params = {
    "n_estimators":[1000], 
    "max_depth":[None], 
    "min_samples_split":[2], 
    "min_samples_leaf":[1]
}


In [10]:
clf = GridSearchCV(estimator=model, param_grid=params,n_jobs=-1, verbose=2)

In [11]:
clf.fit(X, y)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


GridSearchCV(estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'max_depth': [None], 'min_samples_leaf': [1],
                         'min_samples_split': [2], 'n_estimators': [1000]},
             verbose=2)

In [12]:
best = clf.best_estimator_
best

RandomForestRegressor(n_estimators=1000)

In [13]:
clf.best_params_

{'max_depth': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 1000}

In [14]:
y_pred_train = best.predict(X_train)
y_pred_test = best.predict(X_test)
mse_train = mean_squared_error(y_pred_train,y_train)
mse_test = mean_squared_error(y_pred_test,y_test)
mse_train, mse_test

(0.001203799551227722, 0.0011656273476714656)

# Estimacion de resultados 

In [15]:
best.fit(X,y)

RandomForestRegressor(n_estimators=1000)

In [17]:
X_final = test
y_final = best.predict(X_final)
final = pd.DataFrame(X_final['id'])
final['price'] = best.predict(X_final)

In [18]:
final.head()

,id,price
0,0,7.079560
1,1,8.368092
2,2,7.609202
3,3,8.472013
4,4,9.587042


# Exportacion de resultados 

In [20]:
final.to_csv("forest8.csv", index = False, header = True)